# <center>Montar corpus dados de Pesquisa do Instituto Pasteur</center>

In [1]:
## Configurar opções de exibição do pandas para melhor visualizar os dados
from pathlib import Path
from getpass import getpass
from datetime import datetime
from IPython.display import clear_output
import pandas as pd, os, re, sys, time, json, subprocess
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

def find_repo_root(path='.', depth=10):
    ''' 
    Busca o arquivo .git e retorna string com a pasta raiz do repositório
    '''
    # Prevent infinite recursion by limiting depth
    if depth < 0:
        return None
    path = Path(path).absolute()
    if (path / '.git').is_dir():
        return path
    return find_repo_root(path.parent, depth-1)

delay = 10

## Definir a pasta de base do repositório local
base_repo_dir = find_repo_root()

## Sempre construir os caminhos usando os.path.join para compatibilidade WxL
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, '_data', 'in_csv')
folder_data_output = os.path.join(base_repo_dir, '_data', 'out_json')

## Adicionar pastas locais ao sys.path para importar pacotes criados localmente
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from lattes_scrapper import LattesScraper, SoupParser, DictToHDF5, attribute_to_be_non_empty
from scraper_fiocruz_ceara import FiocruzCearaScraper
from json_fle_manager import JSONFileManager as jfm
from scraper_pasteur import PasteurScraper
# from chromedriver_manager import ChromeDriverManager
# from environment_setup import EnvironmentSetup
# from lattes_html_parser import HTMLParser

print(f" Caminho base do repositório: {base_repo_dir}")
print(f"Arquivos de entrada de dados: {jfm.list_json_files(folder_data_input)}")

 Caminho base do repositório: c:\Users\marco\ppgcs
Arquivos de entrada de dados: ['dict_list_complete.json', 'dict_list_discents.json', 'dict_list_docents.json', 'dict_list_temp.json']


In [2]:
# Cria instância da classe ChromeDriverManager e verifica compatibilidade entre versões do Chrome e Chromedriver
# manager = ChromeDriverManager()
# manager.main()

In [3]:
base_url = "https://ceara.fiocruz.br/portal/"
scraper = FiocruzCearaScraper(base_url, find_repo_root())
fioce_data = scraper.scrape_areas()
fioce_data

URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-da-familia/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-e-ambiente/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/
URL pesquisada:
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-digital/pesquisadores-em-saude-digital/


[{'area_name': 'Saúde da Família',
  'area_desc': 'As pesquisas da área avaliam a Atenção Primária à Saúde. São estudos que investigam a atuação das equipes da Estratégia Saúde da Família em diferentes cenários e/ou processos de trabalho e/ou cuidado, em distintos ciclos de vida da população. As temáticas de pesquisa incluem o ensino e as práticas dos profissionais da saúde, saúde mental, condições crônicas, saúde bucal e saúde digital.',
  'area_plats': ['As diversas iniciativas que no momento caracterizam a atuação da Área de Saúde da Família na Fiocruz Ceará consolidam um processo de construção participativo, dinâmico e voltado a garantir respostas aos desafios de fortalecimento e qualificação dessa estratégia que, em virtude de sua magnitude, impõe à sociedade e governos, gigantesco esforços ao efetivo cumprimento de seu propósito. Entende-se pela sua magnitude: a cobertura de mais de 159 milhões de pessoas; mais de 1 milhão de postos de trabalho diretos; cobertura de quase todos o

In [4]:
for i in fioce_data:
    print(i.get('area_pesquisadores'))

['Anya Pimentel Gomes Fernandes Vieira Meyer', 'Ivana Cristina de Holanda Cunha Barreto', 'Luiz Odorico Monteiro de Andrade', 'Márcio Flávio Moura de Araújo', 'Maximiliano Loiola Ponte de Souza', 'Roberto Wagner Júnior Freire de Freitas', 'Sharmênia de Araújo Soares Nuto', 'Vanira Matos Pessoa']
['Ana Cláudia de Araújo Teixeira', 'Fernando Ferreira Carneiro', 'José Luís Passos Cordeiro', 'Margareth Borges Coutinho Gallo']
['Anna Carolina Machado Marinho', 'Carla Freire Celedonio Fernandes', 'Clarissa Romero Teixeira', 'Fábio Miyajima', 'Fernando Braga Stehling Dias', 'Gilvan Pessoa Furtado', 'Jaime Ribeiro Filho', 'João Herminio Martins da Silva', 'Marcos Roberto Lourenzoni', 'Raphael Trevizani', 'Regis Bernardo Brandim Gomes', 'Roberto Nicolete']
['Adriana Costa Bacelo', 'Galba Freire Moita', 'Ivana Cristina De Holanda Cunha Barrêto', 'Luiz Odorico Monteiro de Andrade']


In [5]:
for i in fioce_data:
    print(i.get('area_lines'))

['- Doenças Crônicas e não-transmissíveis, medicina regenerativa', '- Pesquisa Clínica e Ensaios Clínicos', '- Saúde e Gênero, Saúde do Idoso', '- Saúde Perinatal, da Criança e do Adolescente', '- Epidemiologia, métodos estatísticos e quantitativos', '- Políticas Públicas, Planejamento e Gestão em Saúde', '- Gestão de Ciência e Tecnologia em Saúde', '- Promoção da Saúde', '- Avaliação e Economia da Saúde', '- Sociologia, Antropologia, Filosofia e Saúde, Cultura e Sociedade', '- Educação em Ciências e Saúde', '- Políticas Públicas para Saúde em Territórios', '- Tecnologia de informação e comunicação']
['- Vigilância Popular da Saúde, Ambiente e Trabalho', '- Saúde, Agroecologia e Convivência com o Semiárido', '- Impactos dos grandes empreendimentos na saúde', '- Atenção Primária em Saúde, Ambiente e Trabalho', '- Saúde Única/Saúde Planetária (One Health/Global Health)', '- Metodologias de pesquisa crítico-dialógicas e participativas', '- Populações do campo, floresta, águas e áreas urba

In [6]:
linhas_pesquisadores = scraper.scrape_lines()


Pesquisando linhas da área: Biotecnologia...
https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/

Pesquisando linhas da área: Saude Da Familia...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-da-familia/linhas-de-pesquisa/

Pesquisando linhas da área: Saude E Ambiente...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-e-ambiente/linhas-de-pesquisa/

Pesquisando linhas da área: Saude Digital...
https://ceara.fiocruz.br/portal/index.php/pesquisa/saude-digital/pesquisadores-em-saude-digital/


------------------------------------------------------------------------------------------------------------------------
LINHAS DE CADA PESQUISADOR

Anna Carolina Machado Marinho
– Abordagem proteômica para avaliação e estudo de proteoformas
– Expressão, purificação, caracterização estrutural e ensaios de estabilidade de nanocorpos e construtos relacionados para doenças tropicais negligenciadas
– Avaliação e caracterização de biofármacos possibilitand

In [7]:
linhas_pesquisadores

{'Anna Carolina Machado Marinho': ['– Abordagem proteômica para avaliação e estudo de proteoformas',
  '– Expressão, purificação, caracterização estrutural e ensaios de estabilidade de nanocorpos e construtos relacionados para doenças tropicais negligenciadas',
  '– Avaliação e caracterização de biofármacos possibilitando alinhar o desenvolvimento de metodologias analíticas modernas (espectrometria de massas) com PD&I em produtos biológicos'],
 'Carla Freire Celedonio Fernandes': ['– Pesquisa e Desenvolvimento de biofármacos, anticorpos terapêuticos, com ênfase em nanocorpos',
  '– Pesquisa, Desenvolvimento de novas tecnologias para diagnósticos rápidos, miniaturizados, automação, moleculares, biossensores, nanotecnologias para diagnóstico',
  '– Bioprospecção e bioensaios'],
 'Clarissa Romero Teixeira': ['– Imunopatogênese das Leishmanioses',
  '– Resposta imune contra a saliva de vetores artrópodes',
  '– Prospecção e caracterização de novos tratamentos e candidatos a vacina contra a

## Extrair dados do Instituto Pasteur Francês

In [10]:
base_url = "https://research.pasteur.fr"
scraper = PasteurScraper(base_url, find_repo_root())
json_data = scraper.generate_pasteur_report_html()

2024-05-14 19:58:39,247 - INFO - Obtendo os dados do site do Instituto Pasteur, aguarde...
2024-05-14 19:58:40,908 - INFO - 5 seções de dados principais extraídas
2024-05-14 19:58:40,909 - INFO - Obtendo áreas prioritárias de pesquisa, aguarde...
2024-05-14 19:58:53,600 - INFO - 5 áreas prioritárias em pesquisa extraídas
2024-05-14 19:58:53,601 - INFO - Obtendo os centros de referência, aguarde...
2024-05-14 19:58:56,172 - INFO - 5 centros extraídos
2024-05-14 19:58:56,172 - INFO - Obtendo os departamentos, aguarde...
2024-05-14 19:59:21,950 - INFO - 13 departamentos extraídos
2024-05-14 19:59:21,951 - INFO - Obtendo as plataformas, aguarde...
2024-05-14 20:00:53,770 - INFO - 35 plataformas extraídas
2024-05-14 20:00:53,771 - INFO - Relatório concluído!
2024-05-14 20:00:53,772 - INFO - Salvo em: c:\Users\marco\ppgcs\_data\out_json\report_pasteur_research.html


In [11]:
json_data.keys()

dict_keys(['priority_research', 'centers_data', 'heads_centers_data', 'departments_data', 'platforms_data'])

In [12]:
json_data.get('platforms_data')[0]

{'title': 'Collection of the Institut Pasteur (CIP)',
 'head_name': 'Dominique Clermont',
 'link': 'https://research.pasteur.fr/en/team/collection-of-the-institut-pasteur-cip/',
 'aditional_info': {'qte_keywords': '1',
  'qte_members': '8',
  'qte_projects': '4',
  'qte_publication': '105',
  'projects': [{'title': 'ISIDORe – Integrated Services for Infectious Diseases Outbreak Research',
    'head_name': 'Raquel Hurtado Ortiz (PI)',
    'status': 'Ongoing',
    'members': '4 Members',
    'description': 'Representation of MIRRI in ISIDORe The Biological Resource Center (CRBIP) represents the MIRRI infrastructure in ISIDORe. Our activities are focused on service provision (TA), but also in joint research activities. The ISIDORe project intends […]'},
   {'title': 'HoloZcan – Deep Learning Powered Holographic Microscopy for Biothreat Detection on Field',
    'head_name': 'Mariana Ferrari (PI)',
    'status': 'Ongoing',
    'members': '10 Members',
    'description': 'The project is a Re

In [13]:
for i in json_data.get('platforms_data'):
    platform_title = i.get('title')
    print('-'*50)
    print(platform_title.upper())
    try:
        project_list = i.get('aditional_info').get('projects')
        for j in project_list:
            title = j.get('title')
            head = j.get('head_name')
            status = j.get('status')
            description = j.get('description')
            print(f"  [{status}] {title} ({head})")
            print(f"            {description}")
    except:
        try:
            project_list = i.get('aditional_info').get('transversal_projects')
            for j in project_list:
                print(f"[Transversal Project] {j.get('title')}")
        except:
            print(f"            Projetos não encontrados para esta plataforma")

--------------------------------------------------
COLLECTION OF THE INSTITUT PASTEUR (CIP)
  [Ongoing] ISIDORe – Integrated Services for Infectious Diseases Outbreak Research (Raquel Hurtado Ortiz (PI))
            Representation of MIRRI in ISIDORe The Biological Resource Center (CRBIP) represents the MIRRI infrastructure in ISIDORe. Our activities are focused on service provision (TA), but also in joint research activities. The ISIDORe project intends […]
  [Ongoing] HoloZcan – Deep Learning Powered Holographic Microscopy for Biothreat Detection on Field (Mariana Ferrari (PI))
            The project is a Research and Innovation Action, its funding originates from the H2020-EU.3.7.5. – Increase Europe’s resilience to crises and disasters budget line, topic : SU-DRS04-2019-2020 – Chemical, biological, radiological and nuclear (CBRN) […]
  [Ongoing] IS_MIRRI21 – Implementation and Sustainability of Microbial Resource Research Infrastructure for the 21st century (Raquel Hurtado Ortiz (

In [14]:
for i in json_data.get('departments_data'):
    for j in i.get('aditional_info').get('teams'):
        print(j)
        print(f"  {j.get('head_name'):>35}: <b>{j.get('team_name')}</b>")
    print('-'*150)

{'team_name': 'Biological Image Analysis', 'head_name': 'Jean-Christophe Olivo-Marin', 'url': 'https://research.pasteur.fr/en/team/bioimage-analysis/'}
          Jean-Christophe Olivo-Marin: <b>Biological Image Analysis</b>
{'team_name': 'Biology of infection', 'head_name': 'Marc Lecuit', 'url': 'https://research.pasteur.fr/en/team/biology-of-infection/'}
                          Marc Lecuit: <b>Biology of infection</b>
{'team_name': 'Cell Polarity, Migration And Cancer', 'head_name': 'Sandrine Etienne-Manneville', 'url': 'https://research.pasteur.fr/en/team/cell-polarity-migration-and-cancer__trashed/'}
          Sandrine Etienne-Manneville: <b>Cell Polarity, Migration And Cancer</b>
{'team_name': 'Cellular Biology of Microbial Infection', 'head_name': 'Agathe Subtil', 'url': 'https://research.pasteur.fr/en/team/cellular-biology-of-microbial-infection/'}
                        Agathe Subtil: <b>Cellular Biology of Microbial Infection</b>
{'team_name': 'Chromatin and Infection', 'hea

# Testes iniciais

## Recuperar dados quantitativos gerais

In [15]:
scraper = PasteurScraper("https://research.pasteur.fr", base_repo_dir)
main_data = scraper.scrape_main_page_quantitative_data()
print(f"{len(main_data)} seções de dados principais extraídas")
for i,j in main_data.items():
    print(f"{i:>25}: {j}")

5 seções de dados principais extraídas
                    Teams: 145
              Departments: 13
   Platforms & Facilities: 35
     Transversal Projects: 24
        Reference Centers: 23


## Recuperar dados das áreas prioritárias de pesquisa

In [16]:
from pprint import pprint
priority_research = scraper.scrape_priority_scientific_areas()
filepath = os.path.join(folder_data_output,'priority_research.json')
scraper.save_to_json(priority_research, filepath)

print(f"{len(priority_research)} prioridades principais em pesquisa extraídas")
for i in priority_research:
    print(f"  {i.get('area_name'):>50}: {i.get('team_count')}")
print()

Data successfully saved to c:\Users\marco\ppgcs\_data\out_json\priority_research.json
5 prioridades principais em pesquisa extraídas
                        Emerging Infectious Diseases: 78 Teams
                            Antimicrobial resistance: 76 Teams
   Brain connectivity and neurodegenerative diseases: 32 Teams
                                   Cancer Initiative: 49 Teams
            Vaccinology and Immunotherapy initiative: 27 Teams



In [17]:
priority_research

[{'area_name': 'Emerging Infectious Diseases',
  'team_count': '78 Teams',
  'image_url': 'https://research.pasteur.fr/wp-content/uploads/2021/05/research_pasteur-enemerging-infectious-diseasesfrmaladies-infectieuses-emergentes-institutpasteur-i038212-300x135.jpg',
  'details': {'title': 'Emerging Infectious Diseases',
   'directors': [{'name': 'Caroline Demangel',
     'position': 'Head of Structure',
     'role': 'Director'},
    {'name': 'Arnaud Fontanet',
     'position': 'Head of Structure',
     'role': 'Director'}],
   'about': 'Ever since its very early days, the Institut Pasteur has been committed to tackling emerging infections, and its work has left an extraordinary legacy. Many emerging infectious diseases are zoonoses, in which an animal pathogen has crossed the species barrier to cause disease in humans.AIDS, Ebola, SARS, avian flu, chikungunya, Zika, the past fifty years have witnessed an alarming increase in the emergence of new infectious agents into human populations.

## Recuperar dados dos Departamentos do Pasteur

In [18]:
departments_data = scraper.scrape_department_data()
filepath = os.path.join(folder_data_output,'departments_data.json')
scraper.save_to_json(departments_data, filepath)

print(f"{len(departments_data)} departamentos extraídos")
for i in departments_data:
    print(f"  {i.get('head_name'):>25}: {i.get('department_name')}")

Data successfully saved to c:\Users\marco\ppgcs\_data\out_json\departments_data.json
13 departamentos extraídos
          Guillaume Dumenil: Cell Biology & Infection
               Grégory Batt: Computational Biology
           Laure Bally-Cuif: Developmental and Stem Cell Biology
              Romain Koszul: Genomes and Genetics
               Hervé Bourhy: Global Health
           Anne-Lise Giraud: Hearing Institute – Institut Pasteur Center
            Philippe Bousso: Immunology
            Frédéric Barras: Microbiology
             Guilhem Janbon: Mycology
         Pierre-Marie Lledo: Neuroscience
                Lucy Glover: Parasites and Insect Vectors
             Paola Arimondo: Structural Biology and Chemistry
      Jean-Pierre Vartanian: Virology


In [19]:
departments_data

[{'department_name': 'Cell Biology & Infection',
  'head_name': 'Guillaume Dumenil',
  'team_count': '17 Teams',
  'url': 'https://research.pasteur.fr/en/department/cell-biology-infection/',
  'aditional_info': {'qte_keywords': '7',
   'qte_teams': '17',
   'qte_members': '221',
   'qte_projects': 'View all77',
   'qte_publication': '1715',
   'teams': [{'team_name': 'Biological Image Analysis',
     'head_name': 'Jean-Christophe Olivo-Marin',
     'url': 'https://research.pasteur.fr/en/team/bioimage-analysis/'},
    {'team_name': 'Biology of infection',
     'head_name': 'Marc Lecuit',
     'url': 'https://research.pasteur.fr/en/team/biology-of-infection/'},
    {'team_name': 'Cell Polarity, Migration And Cancer',
     'head_name': 'Sandrine Etienne-Manneville',
     'url': 'https://research.pasteur.fr/en/team/cell-polarity-migration-and-cancer__trashed/'},
    {'team_name': 'Cellular Biology of Microbial Infection',
     'head_name': 'Agathe Subtil',
     'url': 'https://research.pas

## Recuperar dados dos Centros do Pasteur

In [20]:
heads_centers_data = scraper.scrape_centers()
centers_data = scraper.scrape_centers_data()
filepath = os.path.join(folder_data_output,'centers_data.json')
scraper.save_to_json(centers_data, filepath)

print(f"{len(centers_data)} centros extraídos")
for i in heads_centers_data:
    print(f"  {i.get('head_name'):>35}: {i.get('title')}")
for i in centers_data:
    print(f"\n{i.get('center_title')} ({len(i.get('teams')):02} times associados)")
    for team in i.get('teams'):
        print(f"  {team.get('head_name'):>35}: {team.get('title')}")

Data successfully saved to c:\Users\marco\ppgcs\_data\out_json\centers_data.json
5 centros extraídos
                       Spencer Shorte: Center for Technological Resources and Research
                         Jean Jaubert: Center for Animal Resources and Research
                      Laurent Essioux: Research and Resource Centre for Scientific Informatics
                           Fay Betsou: Biological Resource Center of Institut Pasteur (CRBIP)
                            Paul Avan: Center for Research and Innovation in Human Audiology

Center for Technological Resources and Research (19 times associados)
                         Milena Hasan: UTechS Single Cell Biomarkers
                   Julia Chamot-Rooke: Mass Spectrometry for Biology
                       Spencer Shorte: Photonic BioImaging (UTechS PBI)
                       Adeline Mallet: Ultrastructural BioImaging
                           Marc Monot: Biomics
                      Patrick England: Molecular Biophys

## Recuperar dados das Plataformas do Pasteur

In [21]:
platforms_data = scraper.scrape_platforms()
filepath = os.path.join(folder_data_output,'platforms_data.json')
scraper.save_to_json(platforms_data, filepath)

print(f"{len(platforms_data)} plataformas extraídas")
for i in platforms_data:
    print(f"  {i.get('head_name'):>25}: {i.get('title')}")

Data successfully saved to c:\Users\marco\ppgcs\_data\out_json\platforms_data.json
35 plataformas extraídas
         Dominique Clermont: Collection of the Institut Pasteur (CIP)
           Alexis Criscuolo: Expertise group: GIPhy – Genome Informatics and Phylogenetics
              Muriel Gugger: Collection of Cyanobacteria
       Raquel Hurtado Ortiz: National Collection of Cultures of Microorganisms
              Emmanuel Roux: ICAReB-biobank : Integrated Collections for Adaptive Research in  Biomedicine
          Mathilde Dusseaux: Human Disease Models core facility
                David Hardy: HistoPathology Core Facility
               Jean Jaubert: Central Animal Facility
       Francina Langa Vives: Mouse Genetics Engineering
              Marta Lindner: Hearing Institute Animal Phenotyping Core Facility
            Sabine Thiberge: Center for the Production and Infection of Anopheles
               Fabrice Agou: Chemogenomic and Biological Screening Platform (PF-CCB)
          

## Recuperar dados dos Times do Pasteur

In [22]:
teams_data = scraper.scrape_teams_data()
print(f"{len(teams_data)} times extraídos")
for i in teams_data:
    print(f"  {i.get('head_name'):>25}: {i.get('title')}")

21 times extraídos
               Fabrice Agou: Chemogenomic and Biological Screening Platform (PF-CCB)
               Pedro Alzari: Structural Microbiology
              Rogerio Amino: Malaria Infection & Immunity
          Benoit Arcangioli: Dynamics of the Genome
             Paola Arimondo: Epigenetic Chemical Biology
             Paola Arimondo: UMR3523 – Chemistry for Life Sciences (Chem4Life)
                  Luc Arnal: Auditory Cognition and Communication
              Sandrine Aros: Metabolomics Core Facility
             Hugues Aschard: Statistical Genetics
              Sylvain Baize: Haemorrhagic Fevers
              Sylvain Baize: Biology of Viral Emerging Infections
           Laure Bally-Cuif: Zebrafish Neurogenetics
           Laure Bally-Cuif: UMR3738 – Developmental and Stem Cell Biology
             Charles Baroud: Physical Microfluidics and Bioengineering
             Jérémie Barral: Neural coding in the auditory system
            Frédéric Barras: Stress Adaptatio

In [23]:
filepath = os.path.join(folder_data_output,'teams_data.json')
scraper.save_to_json(teams_data, filepath)

Data successfully saved to c:\Users\marco\ppgcs\_data\out_json\teams_data.json


In [24]:
centers_data[1]

{'center_title': 'Center for Animal Resources and Research',
 'teams': [{'title': 'Central Animal Facility', 'head_name': 'Jean Jaubert'},
  {'title': 'Center for the Production and Infection of Anopheles',
   'head_name': 'Sabine Thiberge'},
  {'title': 'Mouse Genetics Engineering', 'head_name': 'Francina Langa Vives'},
  {'title': 'Hearing Institute Animal Phenotyping Core Facility',
   'head_name': 'Marta Lindner'},
  {'title': 'Human Disease Models core facility',
   'head_name': 'Mathilde Dusseaux'},
  {'title': 'HistoPathology Core Facility', 'head_name': 'David Hardy'}]}

In [25]:
departments_data[-1]

{'department_name': 'Virology',
 'head_name': 'Jean-Pierre Vartanian',
 'team_count': '24 Teams',
 'url': 'https://research.pasteur.fr/en/department/virology/',
 'aditional_info': {'qte_teams': '24',
  'qte_members': '204',
  'qte_projects': 'View all103',
  'qte_publication': '1953',
  'teams': [{'team_name': 'Arboviruses and Insect Vectors',
    'head_name': 'Anna-Bella Failloux',
    'url': 'https://research.pasteur.fr/en/team/arboviruses-and-insect-vectors/'},
   {'team_name': 'Biology of Viral Emerging Infections',
    'head_name': 'Sylvain Baize',
    'url': 'https://research.pasteur.fr/en/team/biology-of-viral-emerging-infections/'},
   {'team_name': 'HIV, Inflammation and Persistence',
    'head_name': 'Michaela Muller-Trutwin',
    'url': 'https://research.pasteur.fr/en/team/hiv-inflammation-and-persistence/'},
   {'team_name': 'Oncogenic Virus Epidemiology and Pathophysiology',
    'head_name': 'Antoine Gessain',
    'url': 'https://research.pasteur.fr/en/team/oncogenic-virus

In [26]:
platforms_data[-1]

{'title': 'Image Analysis Hub',
 'head_name': 'Jean-Yves Tinevez',
 'link': 'https://research.pasteur.fr/en/team/image-analysis-hub/',
 'aditional_info': {'qte_keywords': '3',
  'qte_members': '8',
  'qte_publication': '61',
  'transversal_projects': [{'title': 'Advanced Light Microscopy initiative'},
   {'title': 'Artificial Intelligence at the Institut Pasteur'}],
  'about': {'title': 'About',
   'content': 'The Image Analysis Hub is an open access, equal accesscore facilitycommitted to offeringbioimage analysis services to the scientific community.What we do.The IAH works mainly by collaborating scientifically with its users, trying to achieve a significant added-value to their Research.Our services are deployed in four activities:Infrastructure for autonomous image analysis.We maintain and offer access to workstations dedicated to and tailored for image analysis software. This includes academic open-source software such as Fiji, Icy, ilastik, QuPath, … and commercial software likeI

## Outros testes

In [27]:
# import requests
# from bs4 import BeautifulSoup
# import json

# # Requisição HTTP para obter o conteúdo da página
# url = "https://ceara.fiocruz.br/portal/index.php/pesquisa/biotecnologia/pesquisadores/"
# response = requests.get(url)

# # Verificação do status da requisição
# if response.status_code == 200:
#     # Conteúdo HTML da página
#     html_content = response.content

#     # Criação do objeto BeautifulSoup
#     soup = BeautifulSoup(html_content, 'lxml')
#     dados_div = {}

#     # Localização das divs com a classe especificada
#     divs = soup.find_all('div', class_="col-sm-12")
#     # divs = soup.find_all('div', style="background-color:#d0d3aa")
#     # divs = soup.find_all('div', style="is-layout-flex wp-container-7 wp-block-columns has-background has-medium-font-size")

#     # Lista para armazenar os dados em JSON
#     dados_json = []

#     # Iteração pelas divs
#     for div in divs:
#         div_pesquisador = div.find('div', class_='is-layout-flex wp-container-7 wp-block-columns has-background has-medium-font-size')
#         # if div_pesquisador:
#         #     if div_pesquisador.get_text():
#         #         print(div_pesquisador.get_text().strip())
#         # Dicionário para armazenar os dados de cada div
#         dados_div = {}

#         # Nome
#         nome_tag = div.find('p', class_='has-black-color has-text-color has-medium-font-size')
#         if nome_tag:
#             nome = nome_tag.text.strip()
#             dados_div['nome'] = nome

#         # Identificação em publicações
#         identificacao_tag = div.find('p', class_='has-small-font-size')
#         if identificacao_tag:
#             texto_identificacao = identificacao_tag.text.strip()
#             if 'Identificação em publicações:' in texto_identificacao:
#                 identificacao = texto_identificacao.split(':')[1].strip()
#                 dados_div['identificacao'] = identificacao

#         # Contato
#         contato_tag = div.find('p', class_='has-small-font-size')
#         if contato_tag:
#             texto_contato = contato_tag.find_next_sibling('p').text.strip()
#             if 'Contato:' in texto_contato:
#                 contato = texto_contato.split(':')[1]
#                 if contato:
#                     dados_div['contato'] = contato

#         # Currículo Lattes
#         curriculo_lattes_tag = div.find('p', class_='has-small-font-size')
#         if curriculo_lattes_tag:
#             try:
#                 link_lattes = curriculo_lattes_tag.find_next_sibling('p').find('a')['href']
#                 if link_lattes:
#                     dados_div['curriculo_lattes'] = link_lattes
#             except:
#                 pass
            
#         # Formação e experiência
#         formacao_experiencia_tag = div.find_all_next('p', class_='has-small-font-size')
#         if formacao_experiencia_tag:
#             texto_formacao_experiencia = '\n'.join([p.text.strip() for p in formacao_experiencia_tag])
#             dados_div['formacao_experiencia'] = texto_formacao_experiencia.split('\n')[1:]

#         # Linhas de pesquisa
#         linha_pesquisa_tag = div.find('h4')
#         if linha_pesquisa_tag:
#             linha_pesquisa_text = linha_pesquisa_tag.find_next_sibling('p').text.strip()
#             if linha_pesquisa_text:
#                 dados_div['linha_pesquisa'] = linha_pesquisa_text

#         # Áreas de pesquisa
#         areas_pesquisa_tag = div.find('h6')
#         if areas_pesquisa_tag:
#             texto_areas_pesquisa = areas_pesquisa_tag.find_next_sibling('p').text.strip()
#             if ' – ' in texto_areas_pesquisa:
#                 areas_pesquisa = texto_areas_pesquisa.split(' – ')
#                 dados_div['areas_pesquisa'] = areas_pesquisa

#         # Adiciona os dados da div à lista
#         dados_json.append(dados_div)

#     # Conversão da lista para JSON
#     json_data = json.dumps(dados_json, ensure_ascii=False, indent=4)

#     # Impressão do JSON formatado
#     print(json_data)

# else:
#     print(f"Erro ao requisitar a página: {response.status_code}")


In [28]:
# dados_div